## `aross-stations-db` demo

In [1]:
import aross_stations_db
version = aross_stations_db.__version__

print(f"Welcome; aross-statations-db v{version} is installed and imported.")

Welcome; aross-statations-db v0.1.dev45+g3d35165.d20240702 is installed and imported.


In [2]:
import leafmap
from ipyleaflet import (
    basemaps,
    projections,
)


START_DATE = "2023-01-01"
END_DATE = "2023-06-01"
# POLYGON = "POLYGON ((-159.32130625160698 69.56469019745796, -159.32130625160698 68.08208920517862, -150.17196253090276 68.08208920517862, -150.17196253090276 69.56469019745796, -159.32130625160698 69.56469019745796))"
API_QUERY_URL = f"http://api:8000/v1/stations?start={START_DATE}&end={END_DATE}"


m = leafmap.Map(
    center=(90, 0),
    zoom=1,
    basemap=basemaps.NASAGIBS.BlueMarble3413,
    crs=projections.EPSG3413.NASAGIBS,

)
m.add_geojson(
    API_QUERY_URL,
    layer_name="ASOS stations",
)

m

Map(center=[90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…